## 1 - Download image chips from Google Earth Engine

This notebook represents the initial step in the application of a methodology for identifying reforestation projects in the Brazilian Atlantic Rainforest. Developed by Felipe Begliomini as part of his MRes dissertation for the AI4ER program at the University of Cambridge, this notebook serves as the foundation for the project.

The primary objective of this notebook is to provide the necessary steps for downloading image chips from Google Earth Engine. The data will be used to train a U-Net model, enabling the identification of reforestation projects within the Brazilian Atlantic Rainforest. Inspiration for this notebook was drawn from two key sources: [Fast(er) Downloads](https://gorelick.medium.com/fast-er-downloads-a2abd512aa26) and the [geemap notebook](https://geemap.org/notebooks/96_image_chips/).

#### Import files

In [ ]:
import ee
from src.data.tools.download_gee import GetImageChips
from src.data.tools.rename_image import rename_chips
import geemap

#### Authenticate and Initialize GEE to use the high-volume endpoint

- [high-volume endpoint](https://developers.google.com/earth-engine/cloud/highvolume)

In [ ]:
# Uncoment the next line if you need to authinticate this device
#ee.Authenticate()

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

#### Import the field data

In [ ]:
# Import the rasterized data in raster format (5m)
ref_raster_ne = ee.Image('users/fnincao/MRes/suzano_nordeste')

ref_raster_se = ee.Image('users/fnincao/MRes/final_sudeste')

# Define the region of Interest
region_ne = ref_raster_ne.geometry()

region_se = ref_raster_se.geometry()

#### Import the points that will be used to create the image chips

In [ ]:
# Import the points for sample the region
points_ne = ee.FeatureCollection('users/fnincao/MRes/sampling_final')\
              .filterBounds(region_ne)
    
points_se = ee.FeatureCollection('users/fnincao/MRes/sampling_final')\
              .filterBounds(region_se)
points_se

#### Generates the images to extract the chips

##### Reference Image

In [ ]:
# Convert the reference image to a binary mask
ref_image_ne = ref_raster_ne\
        .where(ref_raster_ne.gt(0), 1)\
        .unmask(0)\
        .toInt()

ref_image_se = ref_raster_se\
        .where(ref_raster_se.gt(0), 1)\
        .unmask(0)\
        .toInt()

##### Planet Image (NIR / RED / GREEN) (5m)

In [ ]:
# Create a composite from Planet basemap from 2020
planet_image =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
            .filterDate('2020-01-01','2020-12-31')\
            .median()\
            .select('R','G','B','N')

planet_image

##### Planet temporal NDVI image (2016 / 2018 / 2020) (5M)

In [ ]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['N', 'R']).rename('NDVI')
    return image.addBands(ndvi);


# Create a composite from Planet red bands from 2016-2020
planet_red_2016 =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                     .filterDate('2016-01-01', '2016-12-31')\
                     .select(['N','R'])\
                     .map(addNDVI)\
                     .median()\
                     .select('NDVI')

planet_red_2017 =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                     .filterDate('2017-01-01', '2017-12-31')\
                     .select(['N','R'])\
                     .median()
            
planet_red_2018 =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                     .filterDate('2018-01-01', '2018-12-31')\
                     .select(['N','R'])\
                     .map(addNDVI)\
                     .median()\
                     .select('NDVI')

planet_red_2019 =  ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                     .filterDate('2019-01-01', '2019-12-31')\
                     .select(['N','R'])\
                     .map(addNDVI)\
                     .median()\
                     .select('NDVI')

planet_red_2020 = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas")\
                    .filterDate('2020-01-01', '2020-12-31')\
                    .select(['N','R'])\
                    .map(addNDVI)\
                    .median()\
                    .select('NDVI')
           
mosaic_planet_red = planet_red_2016.addBands([planet_red_2018,
                                              planet_red_2020])

##### Sentinel-1 Images (Band-C VH) (10m)

In [ ]:
s1_vh_2016 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2016-01-01', '2016-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

s1_vh_2017 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2017-01-01', '2017-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

s1_vh_2018 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

s1_vh_2019 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2019-01-01', '2019-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

s1_vh_2020 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterDate('2020-01-01', '2020-12-31')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
            .select('VH').median()

mosaic_s1_vh = s1_vh_2016.addBands([s1_vh_2018,
                                    s1_vh_2020])


##### Sentinel-2 Images (Red-Edge) (20m)

In [ ]:
sentinel_image_2022 = ee.ImageCollection('COPERNICUS/S2_SR') \
                        .filterDate('2022-05-01', '2022-09-01') \
                        .select('B5') \
                        .median() 
                   

sentinel_image_2021 = ee.ImageCollection('COPERNICUS/S2_SR') \
                        .filterDate('2021-05-01', '2021-09-01') \
                        .select('B5') \
                        .median()

sentinel_image_2020 = ee.ImageCollection('COPERNICUS/S2_SR') \
                        .filterDate('2020-05-01', '2020-09-01') \
                        .select('B5') \
                        .median()

sentinel_image = sentinel_image_2020.addBands([sentinel_image_2021,
                                               sentinel_image_2022])

##### ALOS/PALSAR-2 Images (Band-L HV) (25m)

In [ ]:
palsar_hv_2016 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                  .filter(ee.Filter.date('2016-01-01', '2016-12-31'))\
                  .select('HV')\
                  .first()

palsar_hv_2017 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                  .filter(ee.Filter.date('2017-01-01', '2017-12-31'))\
                  .select('HV')\
                  .first()


palsar_hv_2018 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                  .filter(ee.Filter.date('2018-01-01', '2018-12-31'))\
                  .select('HV')\
                  .first()

palsar_hv_2019 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                   .filter(ee.Filter.date('2019-01-01', '2019-12-31'))\
                   .select('HV')\
                   .first()


palsar_hv_2020 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')\
                   .filter(ee.Filter.date('2020-01-01', '2020-12-31'))\
                   .select('HV')\
                   .first()

mosaic_palsar_hv = palsar_hv_2016.addBands([palsar_hv_2018,
                                            palsar_hv_2020])

#### Download the Reference images

In [ ]:
%%time
GetImageChips(download_image=ref_image_ne,
              out_resolution=5,
              points=points_ne,
              out_dir='../data/gee_data',
              sulfix='_ref_ne')

In [ ]:
%%time
GetImageChips(download_image=ref_image_se,
              out_resolution=5,
              points=points_se,
              out_dir='../data/gee_data',
              sulfix='_ref')

#### Download Planet images

In [ ]:
%%time
GetImageChips(download_image=planet_image,
              out_resolution=5,
              points=points_ne,
              out_dir='../data/gee_data',
              sulfix='_planet_ne')

In [ ]:
%%time
GetImageChips(download_image=planet_image,
              out_resolution=5,
              points=points_se,
              out_dir='../data/gee_data',
              sulfix='_planet')

#### Download Planet NDVI temporal images

In [ ]:
%%time
GetImageChips(download_image=mosaic_planet_red,
              out_resolution=5,
              points=points_ne,
              out_dir='../data/gee_data',
              sulfix='_ndvi_ne')

In [ ]:
%%time
GetImageChips(download_image=mosaic_planet_red,
              out_resolution=5,
              points=points_se,
              out_dir='../data/gee_data',
              sulfix='_ndvi')

#### Download Sentinel-1 images

In [ ]:
%%time
GetImageChips(download_image=mosaic_s1_vh,
              out_resolution=10,
              points=points_ne,
              out_dir='../data/gee_data',
              sulfix='_s1_ne')

In [ ]:
%%time
GetImageChips(download_image=mosaic_s1_vh,
              out_resolution=10,
              points=points_se,
              out_dir='../data/gee_data',
              sulfix='_s1')

#### Download Sentinel-2 images

In [ ]:
'''%%time
GetImageChips(download_image=sentinel_image,
              out_resolution=20,
              points=points,
              out_dir='../data/gee_data',
              sulfix='_s2'''

#### Download ALOS/PALSAR-2 images

In [ ]:
%%time
GetImageChips(download_image=mosaic_palsar_hv,
              out_resolution=20,
              points=points_ne,
              out_dir='../data/gee_data',
              sulfix='_palsar_ne')

In [ ]:
%%time
GetImageChips(download_image=mosaic_palsar_hv,
              out_resolution=20,
              points=points_se,
              out_dir='../data/gee_data',
              sulfix='_palsar')

#### Rename Images with different string

In [ ]:
rename_chips(sensor='ref')
rename_chips(sensor='planet')
rename_chips(sensor='ndvi')
rename_chips(sensor='s1')
rename_chips(sensor='palsar')